# Smart Parser

Ziel ist ein Parser, der einfache Sätze besser versteht. Eingaben sind gewöhnliche Text-Adventure-Sätze, Ausgaben sind die Commands.

Der Parser benötigt mehrere Stufen um die Syntax zu verstehen, diese mit den erlaubten Verben zu Matchen und die Objekte zu identifizieren.

Verwendet werden:
- SpaCy: Model zur Analyse und Annotation der eingegebenen Sätze. Vermutlich "de_dep_news_trf".
- SentenceTransformer: Zum Matching der Commands, vermutlich mit "paraphrase-multilingual-MiniLM-L12-v2".
- Neo4J: Zum identifizieren der Objekte.

In [1]:
import spacy 
from sentence_transformers import SentenceTransformer, util

# parsing_model_lg = spacy.load("de_core_news_lg")
parsing_model_trf = spacy.load("de_dep_news_trf")
matching_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [2]:
# doc_lg = parsing_model_lg("Nimm den goldenen Schlüssel und öffne die verzauberte Truhe")
doc_trf = parsing_model_trf("Nimm den goldenen Schlüssel und öffne die verzauberte Truhe")

# spacy.displacy.render(doc_lg, style='dep', jupyter=True)
spacy.displacy.render(doc_trf, style='dep', jupyter=True)

## Testsätze

KI-generierte Sätze verschiedener Kategorien inklusive expected outcome.

In [3]:
basic = [
    {
        'sentence': "Nimm den Schlüssel",
        'expected': {'command': 'take', 'objects': ['Schlüssel']}
    },
    {
        'sentence': "Lege das Schwert ab",
        'expected': {'command': 'drop', 'objects': ['Schwert']}
    },
    {
        'sentence': "Geh zur Taverne",
        'expected': {'command': 'visit', 'objects': ['Taverne']}
    },
    {
        'sentence': "Zeige das Inventar",
        'expected': {'command': 'show', 'objects': ['Inventar']}
    },
    {
        'sentence': "Benutze den Schlüssel",
        'expected': {'command': 'use', 'objects': ['Schlüssel']}
    },
    {
        'sentence': "Rede mit dem Wirt",
        'expected': {'command': 'talk', 'objects': ['Wirt']}
    },
    {
        'sentence': "Untersuche die Truhe",
        'expected': {'command': 'look', 'objects': ['Truhe']}
    }
]

trennbar = [
    {
        'sentence': "Nimm den Schlüssel auf",
        'expected': {'command': 'take', 'objects': ['Schlüssel']}
    },
    {
        'sentence': "Wirf den Apfel weg",
        'expected': {'command': 'drop', 'objects': ['Apfel']}
    },
    {
        'sentence': "Lauf zur Taverne",
        'expected': {'command': 'visit', 'objects': ['Taverne']}
    },
    {
        'sentence': "Schau dir das Inventar an",
        'expected': {'command': 'show', 'objects': ['Inventar']}
    },
    {
        'sentence': "Wende den Trank an",
        'expected': {'command': 'use', 'objects': ['Trank']}
    },
    {
        'sentence': "Sprich mit dem Händler",
        'expected': {'command': 'talk', 'objects': ['Händler']}
    },
    {
        'sentence': "Schau dir die Karte an",
        'expected': {'command': 'look', 'objects': ['Karte']}
    }
]

komplex = [
    {
        'sentence': "Nimm den goldenen Schlüssel",
        'expected': {'command': 'take', 'objects': ['goldenen Schlüssel']}
    },
    {
        'sentence': "Lege das schwere eiserne Schwert ab",
        'expected': {'command': 'drop', 'objects': ['schwere eiserne Schwert']}
    },
    {
        'sentence': "Geh in die dunkle Taverne",
        'expected': {'command': 'visit', 'objects': ['dunkle Taverne']}
    },
    {
        'sentence': "Zeige mir meine magischen Gegenstände",
        'expected': {'command': 'show', 'objects': ['magischen Gegenstände']}
    },
    {
        'sentence': "Öffne die verzauberte Truhe mit dem goldenen Schlüssel",
        'expected': {'command': 'use', 'objects': ['verzauberte Truhe', 'goldenen Schlüssel']}
    },
    {
        'sentence': "Sprich mit dem alten Wirt",
        'expected': {'command': 'talk', 'objects': ['alten Wirt']}
    },
    {
        'sentence': "Untersuche das mysteriöse Buch",
        'expected': {'command': 'look', 'objects': ['mysteriöse Buch']}
    }
]

praepositionen = [
    {
        'sentence': "Hole den Schlüssel aus der Truhe",
        'expected': {'command': 'take', 'objects': ['Schlüssel']}  # Truhe ist nur Context
    },
    {
        'sentence': "Lege das Schwert auf den Tisch",
        'expected': {'command': 'drop', 'objects': ['Schwert']}
    },
    {
        'sentence': "Gehe in die Taverne",
        'expected': {'command': 'visit', 'objects': ['Taverne']}
    },
    {
        'sentence': "Schau auf das Inventar",
        'expected': {'command': 'show', 'objects': ['Inventar']}
    },
    {
        'sentence': "Öffne die Tür mit dem Schlüssel",
        'expected': {'command': 'use', 'objects': ['Tür', 'Schlüssel']}
    },
    {
        'sentence': "Rede mit dem Wirt über das Wetter",
        'expected': {'command': 'talk', 'objects': ['Wirt']}  # "Wetter" ist Topic
    },
    {
        'sentence': "Sieh dir die Inschrift an der Wand an",
        'expected': {'command': 'look', 'objects': ['Inschrift']}
    }
]

synonyme = [
    {
        'sentence': "Greif nach dem Schwert",
        'expected': {'command': 'take', 'objects': ['Schwert']}
    },
    {
        'sentence': "Lass das Schwert fallen",
        'expected': {'command': 'drop', 'objects': ['Schwert']}
    },
    {
        'sentence': "Besuche die Taverne",
        'expected': {'command': 'visit', 'objects': ['Taverne']}
    },
    {
        'sentence': "Liste das Inventar auf",
        'expected': {'command': 'show', 'objects': ['Inventar']}
    },
    {
        'sentence': "Verwende den Hebel",
        'expected': {'command': 'use', 'objects': ['Hebel']}
    },
    {
        'sentence': "Frage den Wirt",
        'expected': {'command': 'talk', 'objects': ['Wirt']}
    },
    {
        'sentence': "Betrachte das Gemälde",
        'expected': {'command': 'look', 'objects': ['Gemälde']}
    }
]

schwierig = [
    {
        'sentence': "Schnapp dir den Schlüssel",
        'expected': {'command': 'take', 'objects': ['Schlüssel']}
    },
    {
        'sentence': "Schmeiß das Schwert weg",
        'expected': {'command': 'drop', 'objects': ['Schwert']}
    },
    {
        'sentence': "Mach dass du zur Taverne kommst",
        'expected': {'command': 'visit', 'objects': ['Taverne']}
    },
    {
        'sentence': "Was habe ich alles dabei?",
        'expected': {'command': 'show', 'objects': ['Inventar']}  # Implizit
    },
    {
        'sentence': "Probier mal den Schalter aus",
        'expected': {'command': 'use', 'objects': ['Schalter']}
    },
    {
        'sentence': "Quatsch mal mit dem Händler",
        'expected': {'command': 'talk', 'objects': ['Händler']}
    },
    {
        'sentence': "Guck dir das mal genauer an",
        'expected': {'command': 'look', 'objects': []}  # Kontextabhängig
    }
]

edge_cases = [
    {
        'sentence': "Nimm Schlüssel",
        'expected': {'command': 'take', 'objects': ['Schlüssel']}
    },
    {
        'sentence': "Schwert ablegen",
        'expected': {'command': 'drop', 'objects': ['Schwert']}
    },
    {
        'sentence': "Ich möchte zur Taverne gehen",
        'expected': {'command': 'visit', 'objects': ['Taverne']}
    },
    {
        'sentence': "Kannst du mir das Inventar zeigen?",
        'expected': {'command': 'show', 'objects': ['Inventar']}
    },
    {
        'sentence': "",
        'expected': None
    }
]

all_tests = {
    'basic': basic,
    'trennbar': trennbar,
    'komplex': komplex,
    'praepositionen': praepositionen,
    'synonyme': synonyme,
    'schwierig': schwierig,
    'edge_cases': edge_cases
}

## NLP Processing

In [4]:
# Parser für einzelne Items aus den Testdaten
def add_trf_parsing(items):

    doc = parsing_model_trf(items['sentence'])
    
    # Fügt Verb und Objekte den Testdaten hinzu
    items['rootverb'] = None
    items['objects'] = []
    
    for token in doc:
        # Hauptverb finden (lemma_)
        if token.dep_ == "ROOT":
            items['rootverb'] = token.lemma_
        # Objekte finden
        if token.dep_ in ['obj', 'dobj', 'oa', 'pobj']:
            items['objects'].append(token)

# Command Embedding

In [5]:
# Templates um den Vektorraum je Command zu vergrößern und mehr (als nur die gegebenen) Synonyme zu matchen.
# Imperativformen ergänzen lassen, sind in Model (auf Nachrichten trainiert) nicht vorhanden zu sein. Logisch ;-)
command_verbs = {
    'take': [
        'nehmen', 'holen', 'packen', 'greifen', 'schnappen', 'aufheben', 'raffen',
        'nimm', 'hol', 'pack', 'greif', 'schnapp'
    ],
    'drop': [
        'ablegen', 'werfen', 'lassen', 'fallen lassen', 'wegwerfen', 'schmeißen',
        'leg', 'wirf', 'lass', 'schmeiß'
    ],
    'visit': [
        'gehen', 'laufen', 'besuchen', 'bewegen', 'kommen',
        'geh', 'lauf', 'besuch', 'beweg', 'komm'
    ],
    'show': [
        'zeigen', 'schauen', 'auflisten', 'anschauen', 'ansehen', 'listen',
        'zeig', 'schau', 'list', 'guck', 'sieh'
    ],
    'use': [
        'benutzen', 'verwenden', 'anwenden', 'öffnen', 'betätigen', 'probieren',
        'benutz', 'verwend', 'öffne', 'probier'
    ],
    'talk': [
        'reden', 'sprechen', 'fragen', 'quatschen', 'unterhalten',
        'red', 'rede', 'sprich', 'frag', 'quatsch'
    ],
    'look': [
        'untersuchen', 'betrachten', 'anschauen', 'angucken', 'inspizieren', 'prüfen',
        'untersuch', 'betracht', 'schau', 'guck', 'inspizier'
    ]
}

# Commands embedden
comannd_verb_embedding = {}

for cmd, verbs in command_verbs.items():
    comannd_verb_embedding[cmd] = matching_model.encode(verbs)

In [6]:
# Embedding und Vergleich der Verben mit den Commands
def verb_to_command(items):

    # Abbruch wenn kein Verb gefunden
    if items['rootverb'] is None:
        items['best_command'] = None        
        items['best_sim'] = 0.0
        return

    items['best_command'] = None
    items['best_sim'] = -1

    # Verb embedden
    verb_emb = matching_model.encode(items['rootverb'])   

    for cmd, embs in comannd_verb_embedding.items():

        # Ähnlichkeit vergleichen
        similarities = util.cos_sim(verb_emb, embs)
        max_sim = similarities.max().item()

        # Bestes Ergebnis schreiben wenn > -1 ;-)
        if max_sim > items['best_sim']:
            items['best_sim'] = max_sim
            items['best_command'] = cmd

# Smart Parsing

"Einmal mit alles"

In [7]:
for category, tests in all_tests.items():

    for items in tests:

        add_trf_parsing(items)
        verb_to_command(items)

## AI Analyse

Danke Claude :)

In [ ]:
print("\n" + "="*80)
print(" "*25 + "SMART PARSER TEST REPORT")
print("="*80)

# Per Command Accuracy
from collections import defaultdict

command_stats = defaultdict(lambda: {'total': 0, 'correct': 0})

for category, tests in all_tests.items():
    for item in tests:
        if item['expected'] is None:
            continue

        expected_cmd = item['expected']['command']
        predicted_cmd = item.get('best_command')

        command_stats[expected_cmd]['total'] += 1
        if expected_cmd == predicted_cmd:
            command_stats[expected_cmd]['correct'] += 1

print("\nACCURACY PRO COMMAND:")
print('-'*80)
for cmd, stats in sorted(command_stats.items()):
    acc = 100 * stats['correct'] / stats['total'] if stats['total'] > 0 else 0
    bar = '█' * int(acc / 5)  # Visual bar
    print(f"{cmd:<10} {stats['correct']:>2}/{stats['total']:<2} ({acc:>5.1f}%) {bar}")

print("\n" + "="*80)
print(" "*25 + "ACCURANCY PRO SENTENCE")
print("="*80)

for category, tests in all_tests.items():

    print(f"\nKATEGORIE: {category.upper()}")
    print('-'*80)

    for i, item in enumerate(tests, 1):
        print(f"\n[{i}] {item['sentence']}")
        print(f"    Expected:  {item['expected']}")

        print(f"    Parsed:    verb='{item['rootverb']}', objects={item['objects']}")

        # Prüfen ob expected vorhanden ist
        if item['expected'] is not None:  # ✅
            expected_cmd = item['expected']['command']
            match_icon = '✓' if expected_cmd == item['best_command'] else '✗'
            print(f"    Predicted: {item['best_command']} (score: {item['best_sim']:.3f}) {match_icon}")
        else:
            # Kein Expected → nur Predicted ausgeben
            print(f"    Predicted: {item.get('best_command', 'N/A')} (score: {item.get('best_sim', 0):.3f}) [no expected]")




                         SMART PARSER TEST REPORT

ACCURACY PRO COMMAND:
--------------------------------------------------------------------------------
drop        7/7  (100.0%) ████████████████████
look        0/6  (  0.0%) 
show        4/7  ( 57.1%) ███████████
take        6/7  ( 85.7%) █████████████████
talk        6/6  (100.0%) ████████████████████
use         5/6  ( 83.3%) ████████████████
visit       4/7  ( 57.1%) ███████████

                         ACCURANCY PRO SENTENCE

Kategorie: BASIC
--------------------------------------------------------------------------------

[1] Nimm den Schlüssel
    Expected:  {'command': 'take', 'objects': ['Schlüssel']}
    Parsed:    verb='Nimm', objects=[Schlüssel]
    Predicted: take (score: 0.937) ✓

[2] Lege das Schwert ab
    Expected:  {'command': 'drop', 'objects': ['Schwert']}
    Parsed:    verb='Legen', objects=[Schwert]
    Predicted: drop (score: 0.838) ✓

[3] Geh zur Taverne
    Expected:  {'command': 'visit', 'objects': ['Taver